<a href="https://colab.research.google.com/github/jinyungChoi/CS374/blob/master/notebook/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
PROJECT_PATH = '/content/drive/My Drive/recommendation'

import sys
sys.path.append(PROJECT_PATH)
from os.path import join
import pandas as pd
from sklearn.model_selection import train_test_split
from module.recommender.ncf.dataset import Dataset as NCFDataset

dataset = pd.read_csv(join(PROJECT_PATH, 'data/mason.csv'))
dataset.head()

,Story,User,ChapterCount,TotalCount,ratio
0,7,26,1,42.0,0.023810
1,44,26,1,32.0,0.031250
2,68,26,15,17.0,0.882353
3,155,26,3,18.0,0.166667
4,911,26,8,64.0,0.125000


In [0]:
train, test = train_test_split(dataset, test_size=0.2)

In [4]:
train

,Story,User,ChapterCount,TotalCount,ratio
912040,6021,490910,4,11.0,0.363636
665230,6320,329122,44,44.0,1.000000
1318014,7336,751900,16,16.0,1.000000
153202,6976,53070,1,51.0,0.019608
1182205,6320,674759,34,44.0,0.772727
...,...,...,...,...,...
497191,1203,221330,23,74.0,0.310811
1550913,8705,974616,6,54.0,0.111111
580722,6827,278001,3,148.0,0.020270
308368,7829,117974,5,76.0,0.065789


In [5]:
data = NCFDataset(train=train, test=test, col_user="User", col_item="Story", col_rating="ratio")

/content/drive/My Drive/recommendation/module/recommender/ncf/dataset.py:188: UserWarning: n_neg_test is larger than negative items set size! We will set n_neg as the smallest size: 0
  "n_neg_test is larger than negative items set size! We will set n_neg as the smallest size: %d" % min_num)


In [6]:
from module.recommender.ncf.dataset import Dataset as NCFDataset
from module.recommender.ncf.ncf import NCF

In [7]:
model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=40,
    layer_sizes=[16, 8, 4],
    n_epochs=5,
    batch_size=1000,
    learning_rate=1e-3,
    verbose=2,
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
import numpy as np
len(np.array(data.train_loader(model.n_users)[2]))

1368146

In [9]:
np.array(data.train_loader(model.n_users)[2]).sum()

745847.0

In [0]:
model.fit(data)

training model: neumf
Train on 1231331 samples, validate on 136815 samples
Epoch 1/5
1231331/1231331 - 16s - loss: 0.2944 - val_loss: 0.2701
Epoch 2/5
1231331/1231331 - 6s - loss: 0.2633 - val_loss: 0.2618
Epoch 3/5
1231331/1231331 - 6s - loss: 0.2459 - val_loss: 0.2611
Epoch 4/5
1231331/1231331 - 6s - loss: 0.2099 - val_loss: 0.2708
Epoch 5/5
1231331/1231331 - 6s - loss: 0.1874 - val_loss: 0.2732
{'loss': [0.29436072691117293, 0.2633066071790776, 0.24592364145323828, 0.20994326296770682, 0.1873618628765273], 'val_loss': [0.27010432682212315, 0.2618076088562419, 0.2610932684030091, 0.27077972495002156, 0.27322566723264163]}


In [0]:
import time
start_time = time.time()
prediction = model.predict(test.User, test.Story, is_list=True)
test['prediction'] = prediction
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Story,User,ChapterCount,TotalCount,ratio,prediction
812403,7512,428214,3,52.0,0.057692,0.008763
1612733,8492,1032326,10,115.0,0.086957,0.000678
835579,7808,444030,23,382.0,0.060209,0.991833
1562618,8589,986146,12,73.0,0.164384,0.297094
1167580,6459,660367,1,28.0,0.035714,1.001359


In [0]:
test_time = time.time() - start_time
print("Took {} seconds for prediction.".format(test_time))

Took 10.40811562538147 seconds for prediction.


In [0]:
from module.evaluation.evaludation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

eval_map = map_at_k(test, test, col_prediction='prediction', k=10, col_user="User", col_item="Story", col_rating="ratio")
eval_ndcg = ndcg_at_k(test, test, col_prediction='prediction', k=10, col_user="User", col_item="Story", col_rating="ratio")
eval_precision = precision_at_k(test, test, col_prediction='prediction', k=10, col_user="User", col_item="Story", col_rating="ratio")
eval_recall = recall_at_k(test, test, col_prediction='prediction', k=10, col_user="User", col_item="Story", col_rating="ratio")

In [0]:
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.930346
NDCG:	1.000000
Precision@K:	0.557168
Recall@K:	0.930346


In [0]:
model.save('test')

OSError: ignored

In [10]:
!pip install numpy